In [36]:
import pandas as pd
import altair as alt
import numpy as np
import streamlit as st


In [14]:
stream = pd.read_csv('util.csv', index_col=0)
print(stream.columns.tolist())


['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6']


In [16]:
print(stream.head())

     Unnamed: 1  Unnamed: 2 Unnamed: 3             Unnamed: 4  Unnamed: 5  \
NaN         NaN       Dates        Day  Net Consumption (kWh)         NaN   
NaN         1.0  2024-01-01          1                   11.0         NaN   
NaN         2.0  2024-01-02          2                   11.0         NaN   
NaN         3.0  2024-01-03          3                   11.0         NaN   
NaN         4.0  2024-01-04          4                   11.0         NaN   

     Unnamed: 6  
NaN         NaN  
NaN         NaN  
NaN         NaN  
NaN         NaN  
NaN         NaN  


In [17]:
stream = pd.read_csv('util.csv', index_col=0, skiprows=1)
print(stream.head())
print(stream.columns.tolist())

     Unnamed: 1       Dates  Day  Net Consumption (kWh)  Unnamed: 5  \
NaN           1  2024-01-01    1                   11.0         NaN   
NaN           2  2024-01-02    2                   11.0         NaN   
NaN           3  2024-01-03    3                   11.0         NaN   
NaN           4  2024-01-04    4                   11.0         NaN   
NaN           5  2024-01-05    5                   11.0         NaN   

     Unnamed: 6  
NaN         NaN  
NaN         NaN  
NaN         NaN  
NaN         NaN  
NaN         NaN  
['Unnamed: 1', 'Dates', 'Day', 'Net Consumption (kWh)', 'Unnamed: 5', 'Unnamed: 6']


In [18]:
streamclean = stream.drop(columns=['Unnamed: 1', 'Unnamed: 5', 'Unnamed: 6'])
print(streamclean.head())
print(streamclean.columns.tolist())

          Dates  Day  Net Consumption (kWh)
NaN  2024-01-01    1                   11.0
NaN  2024-01-02    2                   11.0
NaN  2024-01-03    3                   11.0
NaN  2024-01-04    4                   11.0
NaN  2024-01-05    5                   11.0
['Dates', 'Day', 'Net Consumption (kWh)']


In [19]:
# Make sure 'Dates' is datetime type
streamclean['Dates'] = pd.to_datetime(streamclean['Dates'])

# Extract month names for faceting
streamclean['Month'] = streamclean['Dates'].dt.strftime('%B')  # e.g., 'January'

# Show first 5 rows to check
print(streamclean[['Dates', 'Month']].head())

         Dates    Month
NaN 2024-01-01  January
NaN 2024-01-02  January
NaN 2024-01-03  January
NaN 2024-01-04  January
NaN 2024-01-05  January


In [20]:
# Reset index to get rid of that NaN index column showing up
streamclean = streamclean.reset_index(drop=True)

# Show the first 5 rows again
print(streamclean[['Dates', 'Month']].head())

       Dates    Month
0 2024-01-01  January
1 2024-01-02  January
2 2024-01-03  January
3 2024-01-04  January
4 2024-01-05  January


In [21]:
import altair as alt

chart = alt.Chart(streamclean).mark_line(point=True).encode(
    x=alt.X('Day:Q', title='Day of Month'),
    y=alt.Y('Net Consumption (kWh):Q', title='Net Consumption (kWh)'),
    tooltip=[
        alt.Tooltip('Dates:T', title='Date'),
        alt.Tooltip('Net Consumption (kWh):Q', title='Consumption (kWh)'),
    ]
).properties(
    width=600,
    height=300,
    title='Daily Net Consumption'
)

chart.display()


alt.Chart(...)

In [28]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

streamclean['Month'] = pd.Categorical(streamclean['Month'], categories=month_order, ordered=True)

In [29]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

streamclean['Month'] = pd.Categorical(streamclean['Month'], categories=month_order, ordered=True)

In [38]:
faceted_chart = alt.Chart(streamclean).mark_line(point=True).encode(
    x=alt.X('Day:Q', title='Day of Month'),
    y=alt.Y('Net Consumption (kWh):Q', title='Net Consumption (kWh)'),
    tooltip=[
        alt.Tooltip('Dates:T', title='Date'),
        alt.Tooltip('Net Consumption (kWh):Q', title='Consumption (kWh)'),
    ]
).add_params(zoom
).properties(
    width=250,
    height=150
).facet(
    column=alt.Column('Month:N', header=alt.Header(labelAngle=0, labelFontSize=12))
).resolve_scale(
    y='shared'  # share y-axis across charts to compare easily
)

faceted_chart.display()

alt.FacetChart(...)

In [23]:
faceted_chart = alt.Chart(streamclean).mark_line(point=True).encode(
    x=alt.X('Day:Q', title='Day of Month', axis=alt.Axis(tickCount=10)),
    y=alt.Y('Net Consumption (kWh):Q', title='Net Consumption (kWh)'),
    tooltip=[
        alt.Tooltip('Dates:T', title='Date'),
        alt.Tooltip('Net Consumption (kWh):Q', title='Consumption (kWh)'),
    ]
).properties(
    width=350,   # wider charts for better spacing
    height=200,
    title='Daily Net Consumption by Month'
).facet(
    column=alt.Column('Month:N', header=alt.Header(labelAngle=0, labelFontSize=14))
).resolve_scale(
    y='shared'
)

faceted_chart.display()

alt.FacetChart(...)

In [31]:
print(sorted(streamclean['Month'].unique()))

['April', 'August', 'December', 'February', 'January', 'July', 'June', 'March', 'May', 'November', 'October', 'September']


In [35]:
import altair as alt

zoom = alt.selection_interval(encodings=['x'])

faceted_chart = alt.Chart(streamclean).mark_line(point=True).encode(
    x=alt.X('Day:Q', title='Day of Month', axis=alt.Axis(tickCount=10)),
    y=alt.Y('Net Consumption (kWh):Q', title='Net Consumption (kWh)'),
    tooltip=[
        alt.Tooltip('Dates:T', title='Date'),
        alt.Tooltip('Net Consumption (kWh):Q', title='Consumption (kWh)'),
    ]
).add_params(
    zoom
).properties(
    width=350,
    height=200,
    title='Daily Net Consumption by Month (Zoom & Pan Enabled)'
).facet(
    column=alt.Column('Month:N', sort=month_order, header=alt.Header(labelAngle=0, labelFontSize=14))
).resolve_scale(
    y='shared',
    x='shared'
)

faceted_chart.display()

alt.FacetChart(...)

In [39]:
import os
print(os.getcwd())

/Users/oma/Untitled Folder/July12025
